#Importando Bibliotecas

In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.impute import KNNImputer
from IPython.core.display import display, HTML

##Extraindo Dataset

In [2]:
data_path = '/content/sample_data/PEDE 2024 - DATATHON.xlsx'


In [3]:
# Leitura das abas do arquivo Excel
df_2022 = pd.read_excel(data_path, sheet_name='PEDE2022')
df_2023 = pd.read_excel(data_path, sheet_name='PEDE2023')
df_2024 = pd.read_excel(data_path, sheet_name='PEDE2024')

In [4]:
df_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1156 entries, 0 to 1155
Data columns (total 50 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   RA                     1156 non-null   object        
 1   Fase                   1156 non-null   object        
 2   INDE 2024              1092 non-null   object        
 3   Pedra 2024             1092 non-null   object        
 4   Turma                  1156 non-null   object        
 5   Nome Anonimizado       1156 non-null   object        
 6   Data de Nasc           1156 non-null   datetime64[ns]
 7   Idade                  1156 non-null   int64         
 8   Gênero                 1156 non-null   object        
 9   Ano ingresso           1156 non-null   int64         
 10  Instituição de ensino  1155 non-null   object        
 11  Pedra 20               191 non-null    object        
 12  Pedra 21               264 non-null    object        
 13  Ped

In [5]:
# Adicionar coluna Ano Letivo
df_2022["Ano Letivo"] = 2022
df_2023["Ano Letivo"] = 2023
df_2024["Ano Letivo"] = 2024

# Renomeação de colunas para padronização
df_2022 = df_2022.rename(columns={
    "Idade 22": "Idade",
    "Pedra 22": "Pedra",
    "INDE 22": "INDE",
    "Matem": "Mat",
    "Portug": "Por",
    "Inglês": "Ing",
    "Fase ideal": "Fase Ideal",
    "Defas": "Defasagem"
})

df_2023 = df_2023.rename(columns={
    "Nome Anonimizado": "Nome",
    "Pedra 2023": "Pedra",
    "Fase Ideal": "Fase Ideal",
    "INDE 2023": "INDE"

})

df_2024 = df_2024.rename(columns={
    "Nome Anonimizado": "Nome",
    "Pedra 2024": "Pedra",
    "Fase Ideal": "Fase Ideal",
    "INDE 2024": "INDE"
})


##Juntando os três anos letivos 2022 a 2024

In [6]:
# Seleção das colunas padronizadas
colunas_selecionadas = [
    "Ano Letivo","RA","Fase","Turma", "Nome", "Idade", "Gênero", "Ano ingresso",
    "Instituição de ensino", "Pedra","INDE", "Nº Av","IAA", "IEG", "IPS",
    "IDA", "IPV", "IAN", "IPP", "Mat", "Por", "Ing", "Fase Ideal", "Defasagem"
]

# Garantir que todas as colunas existam, preenchendo com NaN caso não existam
def selecionar_colunas(df, colunas):
    for col in colunas:
        if col not in df.columns:
            df[col] = pd.NA  # Preencher com NaN
    return df[colunas]

df_2022 = selecionar_colunas(df_2022, colunas_selecionadas)
df_2023 = selecionar_colunas(df_2023, colunas_selecionadas)
df_2024 = selecionar_colunas(df_2024, colunas_selecionadas)

# Concatenar os DataFrames
merged_df = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)

# Ordenar pelo Ano de Ingresso
merged_df = merged_df.sort_values(by="Ano ingresso").reset_index(drop=True)

# Converter colunas numéricas e substituir valores inválidos por NaN
colunas_numericas = ["INDE", "IAA", "IEG", "IPS", "IDA", "IPV", "IAN", "IPP", "Mat", "Por", "Ing"]
for col in colunas_numericas:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Exibir as primeiras linhas do DataFrame final
print(merged_df.tail())


      Ano Letivo       RA  Fase           Turma        Nome Idade     Gênero  \
3025        2024  RA-1316  ALFA  ALFA I - G2/G3  Aluno-1316     9   Feminino   
3026        2024  RA-1315  ALFA  ALFA H - G0/G1  Aluno-1315     7  Masculino   
3027        2024  RA-1314  ALFA  ALFA H - G0/G1  Aluno-1314     7   Feminino   
3028        2024  RA-1312  ALFA  ALFA H - G0/G1  Aluno-1312     9  Masculino   
3029        2024  RA-1362  ALFA  ALFA R - G0/G1  Aluno-1362     7  Masculino   

      Ano ingresso Instituição de ensino     Pedra  ...    IPS   IDA  \
3025          2024               Pública  Ametista  ...  7.510  7.75   
3026          2024               Pública   Topázio  ...  7.510  9.00   
3027          2024               Pública   Topázio  ...  7.510  9.00   
3028          2024               Pública  Ametista  ...  7.510  6.50   
3029          2024               Pública  Ametista  ...  5.635  8.50   

           IPV   IAN    IPP   Mat   Por  Ing            Fase Ideal  Defasagem  
3025  

<ipython-input-6-e74aa009d1e1>:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)


In [7]:
# Ajustar para exibir todas as colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

merged_df.iloc[1:10]

,Ano Letivo,RA,Fase,Turma,Nome,Idade,Gênero,Ano ingresso,Instituição de ensino,Pedra,INDE,Nº Av,IAA,IEG,IPS,IDA,IPV,IAN,IPP,Mat,Por,Ing,Fase Ideal,Defasagem
1,2022,RA-112,4,A,Aluno-112,15,Menina,2016,Rede Decisão,Ametista,7.836000,4.0,7.9,7.8,6.30,8.5,8.833,5.0,NaN,9.0,7.5,9.1,Fase 5 (1º EM),-1
2,2022,RA-106,4,A,Aluno-106,15,Menina,2016,Rede Decisão,Ametista,7.973000,4.0,9.2,8.0,9.40,7.4,8.375,5.0,NaN,7.8,7.2,7.2,Fase 5 (1º EM),-1
3,2022,RA-105,4,A,Aluno-105,17,Menino,2016,Rede Decisão,Ágata,6.846000,4.0,7.9,8.2,5.60,6.7,7.417,2.5,NaN,7.2,6.2,6.8,Fase 7 (3º EM),-3
4,2022,RA-267,3,I,Aluno-267,15,Menina,2016,Escola Pública,Ágata,6.012000,4.0,10.0,7.4,5.00,3.3,6.292,5.0,NaN,2.3,4.8,2.7,Fase 5 (1º EM),-2
5,2023,RA-40,FASE 6,6A,Aluno-40,17,Feminino,2016,Privada - Programa de Apadrinhamento,Ametista,7.169975,4.0,7.5,8.2,7.52,5.9,7.755,5.0,7.96875,6.2,5.7,5.8,Fase 7 (3° EM),-1
6,2022,RA-93,5,L,Aluno-93,15,Menina,2016,Rede Decisão,Topázio,8.941000,4.0,8.3,9.7,8.10,8.5,9.083,10.0,NaN,8.8,8.3,8.4,Fase 5 (1º EM),0
7,2022,RA-92,5,L,Aluno-92,16,Menina,2016,Rede Decisão,Ágata,6.957000,4.0,9.2,7.1,5.00,7.4,6.667,5.0,NaN,7.8,5.5,8.8,Fase 6 (2º EM),-1
8,2022,RA-91,5,L,Aluno-91,16,Menina,2016,Rede Decisão,Ágata,6.645000,4.0,9.2,5.7,5.60,6.6,7.125,5.0,NaN,5.5,5.7,8.5,Fase 6 (2º EM),-1
9,2023,RA-193,FASE 4,4A,Aluno-193,13,Masculino,2016,Privada - Programa de Apadrinhamento,Ametista,7.718567,4.0,8.3,9.4,2.52,6.5,8.170,10.0,8.12500,6.0,6.3,7.3,Fase 3 (7° e 8° ano),1


In [8]:
def limpar_idade(valor):
    if isinstance(valor, str) and '/' in valor:
        valor = re.sub(r'^[^/]+/|/[^/]+$', '', valor)  # Mantém apenas o valor entre as barras
    return pd.to_numeric(valor, errors='coerce')  # Converte para número, substituindo erros por NaN

# Aplica a função à coluna "Idade"
merged_df["Idade"] = merged_df["Idade"].astype(str).apply(limpar_idade)

# Substitui os NaN pela mediana, garantindo que a coluna seja numérica antes disso
mediana_idade = merged_df["Idade"].dropna().median()  # Calcula a mediana ignorando os NaN
merged_df["Idade"] = merged_df["Idade"].fillna(mediana_idade).astype(int)


In [9]:
# Corrigir a coluna Idade
# Lista de colunas numéricas que devem ter uma casa decimal
colunas_numericas = ["INDE", "Nº Av", "IAA", "IEG", "IPS", "IDA", "IPV", "IAN", "IPP", "Mat", "Por", "Ing"]

# Converter para float e arredondar para uma casa decimal
merged_df[colunas_numericas] = merged_df[colunas_numericas].astype(float).round(1)

# Ajustar a coluna "Fase Ideal"
merged_df.loc[merged_df["Fase Ideal"].str.contains("ALFA", na=False, case=False), "Fase Ideal"] = "Fase 0 "
merged_df["Fase Ideal"] = merged_df["Fase Ideal"].astype(str).str.extract(r"^([^(]+)")

# Substituir "ALFA" por "0" na coluna "Fase Ideal"
merged_df.loc[merged_df["Fase"].str.contains("ALFA", na=False, case=False), "Fase"] = "0"

# Remover todas as letras e caracteres especiais da coluna "Fase", mantendo apenas números
merged_df["Fase"] = merged_df["Fase"].astype(str).apply(lambda x: "".join(re.findall(r"\d+", x)))

In [10]:
# Preencher valores nulos na coluna "Instituição de ensino" com "Não informado"
merged_df["Instituição de ensino"] = merged_df["Instituição de ensino"].fillna("Não informado")

# Se "IPS" for nulo e "Nº Av" for 0, então "IPS" recebe 0
merged_df.loc[merged_df["IPS"].isna() & (merged_df["Nº Av"] == 0), "IPS"] = 0

# Se "IPP" for nulo e "Nº Av" for 0, então "IPS" recebe 0
merged_df.loc[merged_df["IPP"].isna() & (merged_df["Nº Av"] == 0), "IPP"] = 0

In [11]:
# Ajustar para exibir todas as colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

merged_df.iloc[1:10]

,Ano Letivo,RA,Fase,Turma,Nome,Idade,Gênero,Ano ingresso,Instituição de ensino,Pedra,INDE,Nº Av,IAA,IEG,IPS,IDA,IPV,IAN,IPP,Mat,Por,Ing,Fase Ideal,Defasagem
1,2022,RA-112,4,A,Aluno-112,15,Menina,2016,Rede Decisão,Ametista,7.8,4.0,7.9,7.8,6.3,8.5,8.8,5.0,NaN,9.0,7.5,9.1,Fase 5,-1
2,2022,RA-106,4,A,Aluno-106,15,Menina,2016,Rede Decisão,Ametista,8.0,4.0,9.2,8.0,9.4,7.4,8.4,5.0,NaN,7.8,7.2,7.2,Fase 5,-1
3,2022,RA-105,4,A,Aluno-105,17,Menino,2016,Rede Decisão,Ágata,6.8,4.0,7.9,8.2,5.6,6.7,7.4,2.5,NaN,7.2,6.2,6.8,Fase 7,-3
4,2022,RA-267,3,I,Aluno-267,15,Menina,2016,Escola Pública,Ágata,6.0,4.0,10.0,7.4,5.0,3.3,6.3,5.0,NaN,2.3,4.8,2.7,Fase 5,-2
5,2023,RA-40,6,6A,Aluno-40,17,Feminino,2016,Privada - Programa de Apadrinhamento,Ametista,7.2,4.0,7.5,8.2,7.5,5.9,7.8,5.0,8.0,6.2,5.7,5.8,Fase 7,-1
6,2022,RA-93,5,L,Aluno-93,15,Menina,2016,Rede Decisão,Topázio,8.9,4.0,8.3,9.7,8.1,8.5,9.1,10.0,NaN,8.8,8.3,8.4,Fase 5,0
7,2022,RA-92,5,L,Aluno-92,16,Menina,2016,Rede Decisão,Ágata,7.0,4.0,9.2,7.1,5.0,7.4,6.7,5.0,NaN,7.8,5.5,8.8,Fase 6,-1
8,2022,RA-91,5,L,Aluno-91,16,Menina,2016,Rede Decisão,Ágata,6.6,4.0,9.2,5.7,5.6,6.6,7.1,5.0,NaN,5.5,5.7,8.5,Fase 6,-1
9,2023,RA-193,4,4A,Aluno-193,13,Masculino,2016,Privada - Programa de Apadrinhamento,Ametista,7.7,4.0,8.3,9.4,2.5,6.5,8.2,10.0,8.1,6.0,6.3,7.3,Fase 3,1


##Ajustando indicadores IPP , IEG ,Ing,Por,Mat

In [12]:



# Criar uma função para calcular IPP

merged_df.loc[merged_df["INDE"] == "INCLUIR", "INDE"] = None
merged_df.loc[merged_df["Pedra"] == "INCLUIR", "Pedra"] = None

def calcular_ipp(row):
    if pd.isna(row["IPP"]) and not pd.isna(row["INDE"]):
        if row["Fase"] in ["0","1", "2", "3", "4", "5", "6", "7"]:
            return (row["INDE"] - (row["IAN"] * 0.1 + row["IDA"] * 0.2 + row["IEG"] * 0.2 + row["IAA"] * 0.1 + row["IPS"] * 0.1 + row["IPV"] * 0.2)) / 0.1
        else:
            return np.nan  # Não calcular para Fase 8 ou valores inválidos
    return row["IPP"]  # Manter valores existentes

# Aplicar a função para preencher IPP
merged_df["IPP"] = merged_df.apply(calcular_ipp, axis=1)

def calcular_ieg(row):
    if pd.isna(row["IEG"]) and not pd.isna(row["INDE"]):
        if row["Fase"] in ["0","1", "2", "3", "4", "5", "6", "7"]:
            return (row["INDE"] - (row["IAN"] * 0.1 + row["IDA"] * 0.2 + row["IPP"] * 0.1 + row["IAA"] * 0.1 + row["IPS"] * 0.1 + row["IPV"] * 0.2)) / 0.2
        else:
            return np.nan  # Não calcular para Fase 8 ou valores inválidos
    return row["IEG"]  # Manter valores existentes

# Aplicar a função para preencher IPP
merged_df["IEG"] = merged_df.apply(calcular_ieg, axis=1)


# Exibir as primeiras linhas corrigidas
# Ajustar para exibir todas as colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [13]:
merged_df["Nº Av"] = merged_df["Nº Av"].fillna(0)

In [14]:
# Verificar se IDA, Mat e Por não são nulos e Ing é nulo
mask1 = merged_df["Ing"].isna() & merged_df["IDA"].notna() & merged_df["Mat"].notna() & merged_df["Por"].notna()

# Calcular Ing usando a fórmula inversa do IDA
merged_df.loc[mask1, "Ing"] = merged_df["IDA"] * 3 - merged_df["Mat"] - merged_df["Por"]

In [15]:
# Verificar se IDA, Mat e Por não são nulos e Ing é nulo
mask2 = merged_df["Mat"].isna() & merged_df["IDA"].notna() & merged_df["Ing"].notna() & merged_df["Por"].notna()

# Calcular Ing usando a fórmula inversa do IDA
merged_df.loc[mask2, "Mat"] = merged_df["IDA"] * 3 - merged_df["Ing"] - merged_df["Por"]

In [16]:
# Verificar se IDA, Mat e Por não são nulos e Ing é nulo
mask3 = merged_df["Por"].isna() & merged_df["IDA"].notna() & merged_df["Ing"].notna() & merged_df["Mat"].notna()

# Calcular Ing usando a fórmula inversa do IDA
merged_df.loc[mask3, "Por"] = merged_df["IDA"] * 3 - merged_df["Ing"] - merged_df["Mat"]

In [17]:
# Verificar se IDA, Mat e Por não são nulos e Ing é nulo
mask4 = merged_df["Mat"].isna() & merged_df["IDA"].notna() & merged_df["Ing"].notna() & merged_df["Mat"].notna()

# Calcular Ing usando a fórmula inversa do IDA
merged_df.loc[mask4, "Por"] = merged_df["IDA"] * 3 - merged_df["Ing"] - merged_df["Mat"]

In [18]:
# Se "IDA" for 0 e "Mat", "Ing" e "Por" forem nulos, atribuir 0 a essas colunas
cols_to_fill = ["Mat", "Ing", "Por"]
merged_df.loc[(merged_df["IDA"] == 0) & merged_df[cols_to_fill].isna().all(axis=1), cols_to_fill] = 0

In [19]:
merged_df.loc[(merged_df["IDA"] == merged_df["Por"]) & merged_df[["Mat", "Ing"]].isna().all(axis=1), ["Mat", "Ing"]] = merged_df["IDA"]
merged_df.loc[(merged_df["IDA"] == merged_df["Mat"]) & merged_df[["Por", "Ing"]].isna().all(axis=1), ["Por", "Ing"]] = merged_df["IDA"]


In [20]:

# Preencher valores nulos na coluna "Instituição de ensino" com "Não informado"
merged_df["Instituição de ensino"] = merged_df["Instituição de ensino"].fillna("Não informado")

# Se "IPS" for nulo e "Nº Av" for 0, então "IPS" recebe 0
merged_df.loc[merged_df["IPS"].isna() & (merged_df["Nº Av"] == 0), "IPS"] = 0


In [21]:
#merged_df.iloc[2000:3000]
#filtered_df = merged_df[merged_df["INDE"].isna() | (merged_df["INDE"] == 0) | (merged_df["INDE"] == 0.0)]
filtered_df = merged_df[merged_df["Nome"] == "Aluno-751"]
filtered_df.head()

,Ano Letivo,RA,Fase,Turma,Nome,Idade,Gênero,Ano ingresso,Instituição de ensino,Pedra,INDE,Nº Av,IAA,IEG,IPS,IDA,IPV,IAN,IPP,Mat,Por,Ing,Fase Ideal,Defasagem
1196,2022,RA-751,0,K,Aluno-751,11,Menino,2022,Escola Pública,Quartzo,5.2,2.0,8.5,5.8,7.5,3.8,6.2,5.0,-0.6,2.7,4.9,3.8,Fase 2,-2


In [22]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3030 entries, 0 to 3029
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Ano Letivo             3030 non-null   int64  
 1   RA                     3030 non-null   object 
 2   Fase                   3030 non-null   object 
 3   Turma                  3030 non-null   object 
 4   Nome                   3030 non-null   object 
 5   Idade                  3030 non-null   int64  
 6   Gênero                 3030 non-null   object 
 7   Ano ingresso           3030 non-null   int64  
 8   Instituição de ensino  3030 non-null   object 
 9   Pedra                  2845 non-null   object 
 10  INDE                   2845 non-null   float64
 11  Nº Av                  3030 non-null   float64
 12  IAA                    2865 non-null   float64
 13  IEG                    2954 non-null   float64
 14  IPS                    3024 non-null   float64
 15  IDA 

In [23]:
# Ajustar as colunas float para terem uma única casa decimal
float_columns = merged_df.select_dtypes(include=['float64']).columns
merged_df[float_columns] = merged_df[float_columns].round(1)

## **Estimar os indicadores nulos**

In [24]:
display(HTML("<h2 style='color:red'>Esta abordagem nos permite estimar os valores faltantes com base nos padrões observados nos dados disponíveis. </h2>"))

In [25]:
# Selecionar as colunas relevantes para o INDE
inde_cols = ['IAA', 'IEG', 'IPS', 'IDA', 'IPV', 'IAN', 'IPP']

# Criar um objeto KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Aplicar a imputação
merged_df[inde_cols] = imputer.fit_transform(merged_df[inde_cols])

# Calcular o INDE para valores faltantes
merged_df.loc[merged_df['INDE'].isnull(), 'INDE'] = merged_df[inde_cols].mean(axis=1)


In [26]:
def estimate_subject_grade(row, subject):
    if pd.isnull(row[subject]):
        return row[['Mat', 'Por', 'Ing']].mean()
    return row[subject]

for subject in ['Mat', 'Por', 'Ing']:
    merged_df[subject] = merged_df.apply(lambda row: estimate_subject_grade(row, subject), axis=1)

In [27]:
behavior_cols = ['IEG', 'IPS', 'IDA', 'IAN', 'IPP']
merged_df.loc[merged_df['IPV'].isnull(), 'IPV'] = merged_df[behavior_cols].mean(axis=1)

In [28]:
def estimate_ideal_phase(row):
    if pd.isnull(row['Fase Ideal']):
        years_since_entry = row['Ano Letivo'] - row['Ano ingresso']
        return f"Fase {min(years_since_entry + 1, 8)}"
    return row['Fase Ideal']

merged_df['Fase Ideal'] = merged_df.apply(estimate_ideal_phase, axis=1)


In [29]:
def calculate_lag(row):
    if pd.isnull(row['Defasagem']):
        ideal_phase = int(row['Fase Ideal'].split()[1])
        return row['Fase'] - ideal_phase
    return row['Defasagem']

merged_df['Defasagem'] = merged_df.apply(calculate_lag, axis=1)

In [30]:
# Selecionar apenas colunas numéricas
numeric_cols = merged_df.select_dtypes(include=['number']).columns

# Preencher valores nulos nas colunas numéricas com a média
merged_df[numeric_cols] = merged_df[numeric_cols].fillna(merged_df[numeric_cols].mean())


In [31]:
# Ajustar as colunas float para terem uma única casa decimal
float_columns = merged_df.select_dtypes(include=['float64']).columns
merged_df[float_columns] = merged_df[float_columns].round(1)

**Para preencher os valores nulos na coluna Pedra com base no valor do INDE, podemos usar as faixas de classificação fornecidas:
Quartzo: 2.405 a 5.506
Ágata: 5.506 a 6.868
Ametista: 6.868 a 8.230
Topázio: 8.230 a 9.294**

In [32]:
# Função para classificar a Pedra com base no INDE
def classify_pedra(inde):
    if pd.isnull(inde):  # Caso o valor de INDE seja nulo, retorna NaN
        return None
    elif 2.405 <= inde < 5.506:
        return "Quartzo"
    elif 5.506 <= inde < 6.868:
        return "Ágata"
    elif 6.868 <= inde < 8.230:
        return "Ametista"
    elif 8.230 <= inde <= 9.294:
        return "Topázio"
    else:
        return None

# Preencher os valores nulos na coluna 'Pedra' com base no INDE
merged_df['Pedra'] = merged_df.apply(lambda row: classify_pedra(row['INDE']) if pd.isnull(row['Pedra']) else row['Pedra'], axis=1)

**A lógica verifica se o valor de IEG é menor que 7.5. Se for, atribui "Melhorar entrega". Caso contrário, atribui "Boa entrega".**

In [33]:
# Criar a nova coluna 'Destaque IEG' com base na condição
merged_df['Destaque IEG'] = merged_df['IEG'].apply(lambda x: 'Melhorar entrega' if x < 7.5 else 'Boa entrega')

 **Coluna Destaque IDA com as condições fornecidas:
"Empenhar mais": Quando o valor de IDA for menor que 7.5.
"Boas notas": Quando o valor de IDA for maior ou igual a 7.5.**

In [34]:
# Criar a nova coluna 'Destaque IDA' com base na condição
merged_df['Destaque IDA'] = merged_df['IDA'].apply(lambda x: 'Empenhar mais' if x < 7.5 else 'Boas notas')

**Coluna chamada Destaque IPV com base nos valores da coluna IPV, seguindo as condições:
"Integra-se mais": Quando o valor de IPV é menor que 7.5.
"Boa integração": Quando o valor de IPV é maior ou igual a 7.5**

In [35]:
# Criar a nova coluna 'Destaque IPV' com base na condição
merged_df['Destaque IPV'] = merged_df['IPV'].apply(lambda x: 'Integra-se mais' if x < 7.5 else 'Boa integração')

**Coluna "Atingiu PV", que retorna "Não" quando o valor de IPV é menor que 8.33 e "Sim" quando for maior ou igual a 8.33**

In [36]:
# Criar a nova coluna 'Atingiu PV' com base na condição
merged_df['Atingiu PV'] = merged_df['IPV'].apply(lambda x: 'Não' if x < 8.33 else 'Sim')

In [37]:
def rec_av_condition(inde):
    if inde < 4:
        return "Alocado fase anterior"
    elif 4 <= inde < 5.5:
        return "Manter fase atual"
    elif 5.5 <= inde < 6.8:
        return "Manter atual com bolsa"
    elif 6.8 <= inde < 8.2:
        return "Promovido fase"
    elif inde >= 8.2:
        return "Promovido com bolsa"
    else:
        return None

merged_df['REC AV'] = merged_df['INDE'].apply(rec_av_condition)

In [38]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3030 entries, 0 to 3029
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Ano Letivo             3030 non-null   int64  
 1   RA                     3030 non-null   object 
 2   Fase                   3030 non-null   object 
 3   Turma                  3030 non-null   object 
 4   Nome                   3030 non-null   object 
 5   Idade                  3030 non-null   int64  
 6   Gênero                 3030 non-null   object 
 7   Ano ingresso           3030 non-null   int64  
 8   Instituição de ensino  3030 non-null   object 
 9   Pedra                  3030 non-null   object 
 10  INDE                   3030 non-null   float64
 11  Nº Av                  3030 non-null   float64
 12  IAA                    3030 non-null   float64
 13  IEG                    3030 non-null   float64
 14  IPS                    3030 non-null   float64
 15  IDA 

In [39]:
# Salvar a base final em CSV
merged_df.to_csv("dt_curated_passos_magicos.csv", index=False)
merged_df.to_excel("dt_curated_passos_magicos.xlsx", index=False)
# Exibir as primeiras linhas do DataFrame final
print(merged_df.head())

   Ano Letivo      RA Fase Turma       Nome  Idade  Gênero  Ano ingresso  \
0        2022    RA-1    7     A    Aluno-1     19  Menina          2016   
1        2022  RA-112    4     A  Aluno-112     15  Menina          2016   
2        2022  RA-106    4     A  Aluno-106     15  Menina          2016   
3        2022  RA-105    4     A  Aluno-105     17  Menino          2016   
4        2022  RA-267    3     I  Aluno-267     15  Menina          2016   

  Instituição de ensino     Pedra  INDE  Nº Av   IAA  IEG  IPS  IDA  IPV  IAN  \
0        Escola Pública   Quartzo   5.8    4.0   8.3  4.1  5.6  4.0  7.3  5.0   
1          Rede Decisão  Ametista   7.8    4.0   7.9  7.8  6.3  8.5  8.8  5.0   
2          Rede Decisão  Ametista   8.0    4.0   9.2  8.0  9.4  7.4  8.4  5.0   
3          Rede Decisão     Ágata   6.8    4.0   7.9  8.2  5.6  6.7  7.4  2.5   
4        Escola Pública     Ágata   6.0    4.0  10.0  7.4  5.0  3.3  6.3  5.0   

   IPP  Mat  Por  Ing Fase Ideal  Defasagem      Destaqu